In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

from sklearn.linear_model import LinearRegression #For regression model
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [3]:
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [4]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [5]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [6]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [7]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [8]:
print('training set prep')
train_df=seperate_monthly(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)

training set prep
test set prep


In [9]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [10]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [11]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [12]:
Regresion_model= model = LinearRegression() #Define model object

In [13]:
X_train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
240368,0.639731,0.727273,0.166667,0.179177,0.187916,0.190397,0.148547,0.207811,0.213706,0.156714,...,-0.616870,0.707171,-0.447096,-0.489253,0.249947,0.494100,-0.859800,0.798782,0.333333,1.0000
111052,0.295561,0.409091,0.966667,0.303745,0.766630,0.836525,0.781971,0.765083,0.681119,0.692308,...,0.209300,-0.104948,-0.295843,-0.616327,0.072569,0.479990,0.253609,0.230882,0.583333,1.0000
369754,0.984087,1.000000,0.600000,0.130980,0.048503,0.107133,0.048817,0.064121,0.056783,-0.011473,...,-0.609833,0.710549,-0.396977,-0.446267,0.314854,0.508722,-0.838080,0.725468,0.333333,1.0000
180632,0.480746,0.590909,0.333333,0.075519,0.193736,0.262004,0.217131,0.341591,0.293706,0.275619,...,0.182579,0.265366,-0.045048,0.510841,-0.589700,-0.194202,-0.181168,0.450372,0.916667,0.8750
6890,0.018337,0.090909,0.933333,0.463042,0.788803,0.821260,0.789757,0.804139,0.716084,0.740026,...,-0.557341,0.752289,0.322792,0.059766,0.027027,0.292586,0.333724,0.182157,0.583333,0.9375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73858,0.196570,0.363636,0.200000,0.560204,0.656874,0.631418,0.749027,0.716992,0.639441,0.663364,...,-0.190186,0.033566,-0.663783,-0.127640,-0.299425,0.395587,-0.232540,0.005188,0.750000,0.8750
290249,0.772487,0.818182,0.600000,0.242976,0.340078,0.391896,0.449236,0.471291,0.424615,0.453194,...,0.951693,0.247711,-0.459626,0.052602,-0.207917,0.160723,0.425933,0.445071,0.833333,0.8750
111015,0.295462,0.409091,0.966667,0.181607,0.592295,0.614488,0.683438,0.605946,0.547972,0.594524,...,0.119342,-0.328030,0.472554,0.267534,0.123643,0.229861,0.564443,0.187007,0.416667,1.0000
7290,0.019402,0.090909,0.933333,0.253160,0.836752,0.879822,0.849955,0.851355,0.755245,0.756454,...,-0.096234,0.348409,-0.485282,-0.339367,0.381890,0.743330,-0.060086,-0.197158,0.666667,1.0000


In [14]:
model.fit(X_train_df, y_train_df)

LinearRegression()

In [15]:
predictions=model.predict(X_train_df)

In [16]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [17]:
i=375734
final['index']=0
while i<728338:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,9.731718,375734
1,29.003786,375735
2,9.427837,375736
3,0.946243,375737
4,29.868931,375738
...,...,...
352599,20.496391,728333
352600,11.811154,728334
352601,23.711329,728335
352602,28.855309,728336


In [18]:
final.to_csv('/kaggle/working/regression_wetherprediction.csv',index=False)